# Модель LASER для векторизации предложений

# Установка необходимых пакетов

In [ ]:
import os
os.getcwd()

'/content'

In [ ]:
os.listdir('/root')

['.profile',
 '.bashrc',
 '.local',
 '.tmux.conf',
 '.wget-hsts',
 '.config',
 '.cache',
 '.npm',
 '.jupyter',
 '.ipython',
 '.keras']

In [ ]:
!echo $HOME

/root


In [ ]:
!git clone https://github.com/facebookresearch/LASER.git

Cloning into 'LASER'...
remote: Enumerating objects: 938, done.
remote: Counting objects: 100% (151/151), done.
remote: Compressing objects: 100% (98/98), done.
remote: Total 938 (delta 67), reused 127 (delta 51), pack-reused 787
Receiving objects: 100% (938/938), 2.90 MiB | 23.06 MiB/s, done.
Resolving deltas: 100% (370/370), done.


In [ ]:
# !export LASER="/content/laser"
# %env LASER=./laser
%env LASER=./LASER
!echo ${LASER}

env: LASER=./LASER
./LASER


In [ ]:
!wget --trust-server-names -q https://tinyurl.com/nllblaser2
!wget --trust-server-names -q https://dl.fbaipublicfiles.com/nllb/laser/laser2.spm
!wget --trust-server-names -q https://dl.fbaipublicfiles.com/nllb/laser/laser2.cvocab

In [ ]:
!bash ./LASER/install_external_tools.sh

 - creating directory ./LASER/tools-external
Installing external tools
 - creating directory ./LASER/tools-external/moses-tokenizer/tokenizer
 - download tokenizer/tokenizer.perl
 - download tokenizer/detokenizer.perl
 - download tokenizer/normalize-punctuation.perl
 - download tokenizer/remove-non-printing-char.perl
 - download tokenizer/deescape-special-chars.perl
 - download tokenizer/lowercase.perl
 - download tokenizer/basic-protected-patterns
 - creating directory ./LASER/tools-external/moses-tokenizer/share/nonbreaking_prefixes
 - download share/nonbreaking_prefixes/nonbreaking_prefix.ca
 - download share/nonbreaking_prefixes/nonbreaking_prefix.cs
 - download share/nonbreaking_prefixes/nonbreaking_prefix.de
 - download share/nonbreaking_prefixes/nonbreaking_prefix.el
 - download share/nonbreaking_prefixes/nonbreaking_prefix.en
 - download share/nonbreaking_prefixes/nonbreaking_prefix.es
 - download share/nonbreaking_prefixes/nonbreaking_prefix.fi
 - download share/nonbreaking_pr

In [ ]:
%cd ./LASER/tools-external/
!pwd 

/content/LASER/tools-external
/content/LASER/tools-external


In [ ]:
sh = """
#!/bin/bash

InstallSentencePiece () {
  if [ ! -d sentencepiece-master ] ; then
    echo " - download sentencepiece from github"
    wget https://github.com/google/sentencepiece/archive/master.zip
    unzip master.zip
    /bin/rm master.zip
    if [ ! -s /usr/local/bin/spm_encode ] ; then
      echo " - building code "
      cd sentencepiece-master
      mkdir build
      cd build
      cmake ..
      make -j 10
    fi
  fi
}

InstallSentencePiece
"""
with open('script_sent.sh', 'w') as file:
    file.write(sh)

In [ ]:
!bash ./script_sent.sh

 - download sentencepiece from github
--2023-03-19 14:22:17--  https://github.com/google/sentencepiece/archive/master.zip
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/google/sentencepiece/zip/refs/heads/master [following]
--2023-03-19 14:22:17--  https://codeload.github.com/google/sentencepiece/zip/refs/heads/master
Resolving codeload.github.com (codeload.github.com)... 20.205.243.165
Connecting to codeload.github.com (codeload.github.com)|20.205.243.165|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘master.zip’

master.zip              [       <=>          ]  11.54M  4.26MB/s    in 2.7s    

2023-03-19 14:22:20 (4.26 MB/s) - ‘master.zip’ saved [12103656]

Archive:  master.zip
9ffb33a14c97c512103be0ee74740099660b39aa
   creating: sentencepiece-master/
  

In [ ]:
sh_bpe = """
InstallFastBPE () {
  if [ ! -x fastBPE/fast ] ; then
    echo " - download fastBPE software from github"
    wget https://github.com/glample/fastBPE/archive/master.zip
    unzip master.zip
    /bin/rm master.zip
    mv fastBPE-master fastBPE
    cd fastBPE
    echo " - compiling"
    g++ -std=c++11 -pthread -O3 fastBPE/main.cc -IfastBPE -o fast
    if [ $? -eq 1 ] ; then
      echo "ERROR: compilation failed, please install manually"; exit
    fi
    python setup.py install
  fi
}

InstallFastBPE
"""
with open('script_bpe.sh', 'w') as file:
    file.write(sh_bpe)

In [ ]:
!bash ./script_bpe.sh

 - download fastBPE software from github
--2023-03-19 14:25:34--  https://github.com/glample/fastBPE/archive/master.zip
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/glample/fastBPE/zip/refs/heads/master [following]
--2023-03-19 14:25:35--  https://codeload.github.com/glample/fastBPE/zip/refs/heads/master
Resolving codeload.github.com (codeload.github.com)... 20.205.243.165
Connecting to codeload.github.com (codeload.github.com)|20.205.243.165|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘master.zip’

master.zip              [ <=>                ]   9.46K  --.-KB/s    in 0.008s  

2023-03-19 14:25:35 (1.21 MB/s) - ‘master.zip’ saved [9684]

Archive:  master.zip
036711f8fdc3265d64e8e123a0761be12c5a8e74
   creating: fastBPE-master/
  inflating: fastBPE-master/

In [ ]:
!pip install --upgrade pip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 27.1 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 22.0.4
    Uninstalling pip-22.0.4:
      Successfully uninstalled pip-22.0.4


In [ ]:
# !pip install fairseq==0.12.1
!pip install fairseq
!pip install transliterate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached fairseq-0.12.2.tar.gz (9.6 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached sacrebleu-2.3.1-py3-none-any.whl (118 kB)
  Using cached omegaconf-2.0.6-py3-none-any.whl (36 kB)
  Using cached bitarray-2.7.3-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (269 kB)
  Using cached hydra_core-1.0.7-py3-none-any.whl (123 kB)
  Using cached antlr4_python3_runtime-4.8-py3-none-any.whl
  Using cached colorama-0.4.6-py2.py3-none-any.whl (25 kB)
  Using cached portalocker-2.7.0-py2.py3-none-any.whl (15 kB)
  Created wheel for fairseq: filename=fairseq-0.12.2-cp39-cp39-linux_x86_64.whl size=11179770 sha256=633c98af3799cfb429b063422b86c9cfd533c3241da27cd30bc79326941da53e
  Stored in directory: /root/.cache/pip/wheels/59/35/87/2baf2e4a

In [ ]:
# !sudo apt-get install cmake build-essential pkg-config libgoogle-perftools-dev

Reading package lists... Done
Building dependency tree       
Reading state information... Done
pkg-config is already the newest version (0.29.1-0ubuntu4).
build-essential is already the newest version (12.8ubuntu1.1).
cmake is already the newest version (3.16.3-1ubuntu1.20.04.1).
The following additional packages will be installed:
  libunwind-dev
The following NEW packages will be installed:
  libgoogle-perftools-dev libunwind-dev
0 upgraded, 2 newly installed, 0 to remove and 22 not upgraded.
Need to get 917 kB of archives.
After this operation, 7,995 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/main amd64 libunwind-dev amd64 1.2.1-9build1 [472 kB]
Get:2 http://archive.ubuntu.com/ubuntu focal/main amd64 libgoogle-perftools-dev amd64 2.7-1ubuntu2 [445 kB]
Fetched 917 kB in 1s (1,484 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/sh

In [ ]:
# !cd LASER/tools-external/

In [ ]:
# !git clone https://github.com/google/sentencepiece.git 
# !cd sentencepiece
# % mkdir build
# % cd build
# % cmake ..
# % make -j $(nproc)
# % sudo make install
# % sudo ldconfig -v

In [ ]:
# !echo "Test sample text" > en.txt
# !echo "Additional sample" >> en.txt
!cat en.txt

Test sample text
Additional sample


In [ ]:
# !echo "Тестовый примерный текст" > ru.txt
# !echo "Дополнительный образец" >> ru.txt
# !cat ru.txt

Тестовый примерный текст
Дополнительный образец


In [ ]:
%cd ..
%cd ..

/content


In [ ]:
%pwd

'/content'

In [ ]:
import time

start_time = time.time()

!python3 LASER/source/embed.py \
    --input     5K_en.jaro_lines.txt        \
    --encoder   laser2.pt    \
    --spm-model laser2.spm   \
    --output    emb_en.bin       \
    --verbose

  
print(f'Time: {(time.time() - start_time)/60:.2f} minutes')

2023-03-19 14:33:01,848 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX
2023-03-19 14:33:02,193 | INFO | embed | spm_model: laser2.spm
2023-03-19 14:33:02,193 | INFO | embed | spm_cvocab: laser2.cvocab
2023-03-19 14:33:02,193 | INFO | embed | loading encoder: laser2.pt
2023-03-19 14:33:02,785 | INFO | embed | transfer encoder to GPU
2023-03-19 14:33:11,308 | INFO | preprocess | SPM processing 5K_en.jaro_lines.txt  
2023-03-19 14:33:12,053 | INFO | embed | encoding /tmp/tmp4eto6gd4/spm to emb_en.bin
2023-03-19 14:33:20,359 | INFO | embed | encoded 5000 sentences in 8s
Time: 0.36 minutes


In [ ]:
import time

start_time = time.time()

!python3 LASER/source/embed.py \
    --input     5K_ru.jaro_lines.txt        \
    --encoder   laser2.pt    \
    --spm-model laser2.spm   \
    --output    emb_ru.bin       \
    --verbose

print(f'Time: {(time.time() - start_time)/60:.2f} minutes')

2023-03-19 14:33:23,122 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX
2023-03-19 14:33:23,387 | INFO | embed | spm_model: laser2.spm
2023-03-19 14:33:23,387 | INFO | embed | spm_cvocab: laser2.cvocab
2023-03-19 14:33:23,387 | INFO | embed | loading encoder: laser2.pt
2023-03-19 14:33:23,897 | INFO | embed | transfer encoder to GPU
2023-03-19 14:33:26,589 | INFO | preprocess | SPM processing 5K_ru.jaro_lines.txt  
2023-03-19 14:33:27,451 | INFO | embed | encoding /tmp/tmpx8jtjso1/spm to emb_ru.bin
2023-03-19 14:33:34,832 | INFO | embed | encoded 5000 sentences in 7s
Time: 0.24 minutes


In [ ]:
# read embeddings
import numpy as np
dim = 1024
X = np.fromfile("emb_en.bin", dtype=np.float32, count=-1)
# X is a N x 1024 matrix where N is the number of lines in the text file.                                                                       
X.resize(X.shape[0] // dim, dim)  
print(X.shape)

(5000, 1024)


In [ ]:
Y = np.fromfile("emb_ru.bin", dtype=np.float32, count=-1)
Y.resize(Y.shape[0] // dim, dim)
print(Y.shape)

(5000, 1024)


In [ ]:
!pip install sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 52.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 64.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 kB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 85.9 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=70c4809b2741986833c39518fc1021cd22930d75447950409c291f3e4d0b0807
  Stored in directory: /root/.cache/pip/wheels/71/67/06/162a3760c40d74dd40bc855d527008d26341c2b0ecf3e8e11f
Successfully built sentence-transformers


In [ ]:
from sentence_transformers import util

In [ ]:
sims = []
for en, ru in zip(X, Y):
    sims.append(util.cos_sim(en, ru).item())

In [ ]:
with open('5K_laser_cos_sims.txt', 'w', encoding='utf-8') as to_f:
    for i in sims:
        to_f.write(str(i))
        to_f.write('\n')

In [ ]:
from google.colab import files

files.download('emb_ru.bin')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>